# Домашнее задание «Функции и работа с данными»

## Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:

- оценка 2 и меньше - низкий рейтинг

- оценка 4 и меньше - средний рейтинг

- оценка 4.5 и 5 - высокий рейтинг


Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv') # Получение датафрейма из файла

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
def evaluation(rate):
    """
    В зависимости от оценки функция определяет к какому классу относится фильм
    """
    if rate <= 2:                         # оценка 2 и меньше - низкий рейтинг
        return 'низкий рейтинг'
    elif rate <= 4:                       # оценка 4 и меньше - средний рейтинг
        return 'средний рейтинг'
    elif rate <= 5:                       # оценка 4.5 и 5 - высокий рейтинг
        return 'высокий рейтинг'
    return False


ratings['class'] = ratings['rating'].apply(evaluation) # Передаем в функцию evaluation значения столбца rating. Результаты пишем в столбец class
ratings['class'].value_counts()                   # посмотрим как распределяются фильмы по классам

средний рейтинг    63801
высокий рейтинг    22818
низкий рейтинг     13385
Name: class, dtype: int64

In [10]:
ratings

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг
...,...,...,...,...,...
99999,671,6268,2.5,1065579370,средний рейтинг
100000,671,6269,4.0,1065149201,средний рейтинг
100001,671,6365,4.0,1070940363,средний рейтинг
100002,671,6385,2.5,1070979663,средний рейтинг


## Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

In [14]:
# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] 
    }

data = pd.read_csv('ml-latest-small/keywords.csv')
data

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
...,...,...
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688


In [25]:
# делал задание до вебинара. Так и не понял нужно ли было передавать в функцию всю строку и добавлять столбец на лету,
# как в примере на вебинаре. Или можно оставить так? И в чем тогда принципиальное отличие этого задания от задания 1? Обращение
# к функции такое же.

def region(word, dict_):              # в word передаются значения столбца keyword
    for key, value in dict_.items():  # первый цикл перебирает ключи и значения словаря geo_data
        for el in value:              # второй цикл перебирает элементы для каждого значения словаря (т.е. города) 
            if el in word:            # есть ли город в поисковом запросе, то возвращаем ключ (регион)
                return key
    return 'undefined'
data['region'] = data['keyword'].apply(region, dict_ = geo_data)
data['region'].value_counts()         # посчитаем количество получившихся регионов

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

In [23]:
data.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

 1. В переменную years запишите список из всех годов с 1950 по 2010.

 2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
 
  1. для каждой строки пройдите по всем годам списка years
  2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
  
 3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

 4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [69]:
# в этой задаче использовался датасет ml-25m.zip (size: 250 MB)
# т.к. файлы большие, то в github выгружать не буду. Ссылка на датасет: https://grouplens.org/datasets/movielens/25m/

years = [str(x) for x in range(1950, 2011)] # создаем список из всех годов с 1950 по 2010

movies = pd.read_csv('ml-25m/movies.csv')   # сначала получаем датафрейм с названиями фильмов
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [77]:
# В этой задаче для разнообразия пробовал отправлять в функцию всю строку, но получалось очень очень долго
# Пэтому вернулся к варианту с отправкой столбца

def production_year(title, year):
    """
    Функция перебирает все года с 1950 по 2010 и ищет их в названии фильма. Если год найден, то функция возвращает его значение
    Если год в названии фильма не найден, то функция возвращает 1900 год.
    """
    for y in year:
        if y in title:
            return y
    return '1900'

movies['year'] = movies['title'].apply(production_year, year = years)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [55]:
ratings = pd.read_csv('ml-25m/ratings.csv')              # получаем датафрейм с рейтингами
joined = ratings.merge(movies, on='movieId', how='left') # Объединяем файлы в одну таблицу
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama,1994
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,1993
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War,1995
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance,1952


In [79]:
pd.set_option('display.max_rows', None)         # Сброс ограничений на количество выводимых строк, чтобы увидеть все 62 года 
joined.groupby('year').agg({'rating':'mean'}).sort_values('rating', ascending=False) # считам средний rating для каждого year

,rating
year,
1957,3.999492
1954,3.985693
1972,3.956477
1962,3.929445
1952,3.895315
1974,3.891385
1975,3.883071
1964,3.879177
1951,3.876258


In [ ]:
# Получается лучшие фильмы снимали в основном в 1950-х - 1970-х годах